<a href="https://colab.research.google.com/github/RakhaFS/TA-AMS/blob/main/Sentimen_Analisis_Shell.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##SENTIMEN ANALISIS

#Modul

In [1]:
!pip install transformers datasets evaluate wordcloud scikit-learn nltk
!pip install Sastrawi

import pandas as pd
import numpy as np
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import os
os.environ["WANDB_DISABLED"] = "true"

from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from transformers import DataCollatorWithPadding
from datasets import Dataset
import evaluate
import transformers
print(transformers.__version__)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 4.5 MB/s eta 0:00:00


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


4.53.2


#Load data

In [2]:
# Load Data Train (Udah Bersih, Udah Ada Label)
Train = "https://raw.githubusercontent.com/RakhaFS/TA-AMS/refs/heads/main/Train%20Shell.csv"
Test = "https://raw.githubusercontent.com/RakhaFS/TA-AMS/refs/heads/main/Shell%20Indonesia.csv"

Shell_tr = pd.read_csv(Train)  # Ganti nama sesuai file lo
train = Shell_tr[['favorite_count', 'full_text', 'clean_text', 'sentiment']]  # Pastikan kolom sesuai

# Load Data Test (Data Mentah)
df = pd.read_csv(Test)
df = df[['favorite_count', 'full_text']].dropna().reset_index(drop=True)

print('Data Train sebanyak ', len(train))
print('Data Test sebanyak ', len(df))

Data Train sebanyak  1432
Data Test sebanyak  6228


In [3]:
po = train['sentiment'].str.contains('positive')
nt = train['sentiment'].str.contains('neutral')
ng = train['sentiment'].str.contains('negative')
# Tampilkan hasilnya

jpo = po.sum()
jnt = nt.sum()
jng = ng.sum()
print(f"Jumlah positive: {jpo}")
print(f"Jumlah neutral: {jnt}")
print(f"Jumlah negative: {jng}")

Jumlah positive: 468
Jumlah neutral: 788
Jumlah negative: 176


#Preprocessing

In [4]:
# Preprocessing
factory = StopWordRemoverFactory()
stopword = factory.create_stop_word_remover()

def preprocess(text):
    text = str(text).lower()
    text = re.sub(r'http\S+|www.\S+', '', text)
    text = re.sub(r'@\w+', '', text)
    text = re.sub(r'#\w+', '', text)
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

stop_words = set(stopwords.words('indonesian'))
custom_stopwords = {'yg', 'aja', 'dong', 'nih', 'gk', 'kok', 'sih', 'loh', 'lah', 'nya'}
stop_words.update(custom_stopwords)

def remove_stopwords(text):
    return ' '.join([word for word in text.split() if word not in stop_words])

df['clean_text'] = df['full_text'].apply(preprocess).apply(remove_stopwords)
df.head()

,favorite_count,full_text,clean_text
0,0,Oli Mesin Bensin &amp; Diesel Galon 4L SHELL H...,oli mesin bensin amp diesel galon l shell heli...
1,3,WTS GSX-R 2017 150CC bagus KM 28RB EK-CHAIN Jo...,wts gsxr cc bagus km rb ekchain jok leather be...
2,0,Ternyata beli bensin pake member Shell enak ba...,beli bensin pake member shell enak banget harg...
3,0,@detikcom Kalau gua beli bensin di @Shell buka...,gua beli bensin merugikan negara namanya dagan...
4,0,ranking things i d replace my pee with : 1. es...,ranking things i d replace my pee with es cend...


In [5]:
# Pastikan kolom 'clean_text' dan 'sentiment' sudah ada
assert 'clean_text' in train.columns
assert 'sentiment' in train.columns

# Encode label jadi angka (positive = 2, neutral = 1, negative = 0 misalnya)
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
train['label'] = label_encoder.fit_transform(train['sentiment'])

# Load test data juga, dari shell.csv yang tadi udah di-clean dan belum dilabeli
df_test = df[~df.index.isin(train.index)].reset_index(drop=True)
train.head()

,favorite_count,full_text,clean_text,sentiment,label
0,0,Disini ada yg user SPBU Shell dan payment pake...,disini ada yg user spbu shell dan payment pake...,negative,0
1,19,smg raenek spbu shell ya?,smg raenek spbu shell ya,negative,0
2,1,di serang tuh gada spbu shell kah.. ko kmrn ga...,di serang tuh gada spbu shell kah ko kmrn ga l...,negative,0
3,4,@innovacommunity Info nitro di sekitaran Tange...,innovacommunity info nitro di sekitaran tanger...,negative,0
4,1,@LexWu_13 Taun² kmrn isi bensin shell org pom ...,lexwu taun² kmrn isi bensin shell org pom ny k...,negative,0


#Load Model

In [6]:
model_name = "indobenchmark/indobert-base-p1"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(examples):
    return tokenizer(examples['clean_text'], truncation=True)

train_dataset = Dataset.from_pandas(train[['clean_text', 'label']])
test_dataset = Dataset.from_pandas(df[['clean_text']])

train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Map:   0%|          | 0/1432 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/6228 [00:00<?, ? examples/s]

In [7]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

accuracy = evaluate.load("accuracy")
f1 = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy.compute(predictions=preds, references=labels)["accuracy"],
        "f1": f1.compute(predictions=preds, references=labels, average="macro")["f1"]
    }

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch", # Changed from evaluation_strategy
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    num_train_epochs=7,
    weight_decay=0.01,
    save_total_limit=1,
    logging_dir="./logs",
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=train_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()

pytorch_model.bin:   0%|          | 0.00/498M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


/tmp/ipython-input-7-1143753878.py:27: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.636822,0.739525,0.655567
2,No log,0.270774,0.917598,0.899564
3,No log,0.122011,0.966480,0.958484
4,No log,0.050991,0.985335,0.979556
5,No log,0.030836,0.993017,0.990573
6,0.352400,0.016587,0.995810,0.993457
7,0.352400,0.013630,0.995810,0.993312


TrainOutput(global_step=630, training_loss=0.2864742536393423, metrics={'train_runtime': 6148.2858, 'train_samples_per_second': 1.63, 'train_steps_per_second': 0.102, 'total_flos': 239392098660960.0, 'train_loss': 0.2864742536393423, 'epoch': 7.0})

#Sentiment Analysis dengan BERT

In [10]:
predictions = trainer.predict(test_dataset)
pred_labels = np.argmax(predictions.predictions, axis=-1)
df['sentiment'] = label_encoder.inverse_transform(pred_labels)

# Export CSV hasil
df[['favorite_count', 'full_text', 'clean_text', 'sentiment']].to_csv("hasil_finetune_Shell.csv", index=False)
print("✅ Hasil disimpan ke 'hasil_finetune_Shell.csv'")

✅ Hasil disimpan ke 'hasil_finetune_Shell.csv'


In [11]:
look = df[['full_text','clean_text','sentiment']]
look.head()

,full_text,clean_text,sentiment
0,Oli Mesin Bensin &amp; Diesel Galon 4L SHELL H...,oli mesin bensin amp diesel galon l shell heli...,neutral
1,WTS GSX-R 2017 150CC bagus KM 28RB EK-CHAIN Jo...,wts gsxr cc bagus km rb ekchain jok leather be...,neutral
2,Ternyata beli bensin pake member Shell enak ba...,beli bensin pake member shell enak banget harg...,positive
3,@detikcom Kalau gua beli bensin di @Shell buka...,gua beli bensin merugikan negara namanya dagan...,neutral
4,ranking things i d replace my pee with : 1. es...,ranking things i d replace my pee with es cend...,neutral
